# Simulate Data for One Specific EHR_Field

In [1]:
# !pip install faker

In [2]:
import pandas as pd

ICD_codes = ['E11.1', 'E12.2', 'E13', 'A11', 'R33.3', 'E11.1', 'F12.2', 'G13', 'W11', 'S33.3']
ICD_codes = pd.Series(ICD_codes)
ICD_codes.sample(1).values[0]

'S33.3'

In [3]:
from faker import Faker
fake = Faker()
date = fake.date_between(start_date='today', end_date='+30d')

In [9]:
import numpy as np
import pandas as pd 

L = []
for pid in range(1, 4):
    
    for ecid in range(1, np.random.randint(4, 8)):
        
        if np.random.uniform(0, 1) < 0.2:
            D = {}
            D['P_uid'] = f'P{pid}'
            D['EC_uid'] = f'EC{pid}{ecid}'
            D['Diag_uid'] = None # f'Diag{pid}{ecid}{rid}'
            D['DiagV_val'] = None
            D['DiagDT_val'] = None
            L.append(D)
            
        else:
            for rid in range(1, np.random.randint(2,6)):
                # for vid in range(1, np.random.randint(8, 9)):
                len_v = np.random.randint(0, 9)
                # print(pid, ecid, rid)
                D = {}
                D['P_uid'] = f'P{pid}'
                D['EC_uid'] = f'EC{pid}{ecid}'
                D['Diag_uid'] = f'Diag{pid}{ecid}{rid}'
                D['DiagV_val'] = ICD_codes.sample(1).values[0]
                # D['rec2grn_Vgrn'] = ['aa'] * len_v
                D['DiagDT_val'] = fake.date_between(start_date='today', end_date='+30d')
                
                L.append(D)
            
df = pd.DataFrame(L)
df # df_Diag_2_pid_n_ecid_recid_v_dt


fld = 'DiagV_val'
# fld = 'v' # 'DiagV'
fld = 'DiagDT_val'
df

# ec: encounters; vists.
# select P_uid, EC_uid, Diag_uid, DiagV_val, DiagDT_val from DiagTable where P_uid in (XXX):

,P_uid,EC_uid,Diag_uid,DiagV_val,DiagDT_val
0,P1,EC11,Diag111,E11.1,2023-02-09
1,P1,EC12,Diag121,A11,2023-02-14
2,P1,EC12,Diag122,E11.1,2023-02-23
3,P1,EC12,Diag123,R33.3,2023-02-25
4,P1,EC13,None,None,None
5,P1,EC14,Diag141,F12.2,2023-02-13
6,P1,EC14,Diag142,E12.2,2023-03-01
7,P1,EC14,Diag143,E11.1,2023-02-27
8,P1,EC14,Diag144,G13,2023-02-27
9,P1,EC15,Diag151,G13,2023-02-23


# Choose Fld and Get Grain

In [14]:
# P1
# li_all_ec_one_pat = ['EC11', 'EC12', 'EC13']
P2EC_ecid = ['EC11', 'EC12', 'EC13']

P2EC2Diag_diagid = [['D1', 'D2', 'D3'], ['D2', 'D2', 'D3'], ['D1', 'D9', 'D3']]
P2EC2Diag_diagid = np.array(P2EC2Diag_diagid)
P2EC2Diag_diagid.shape

(3, 3)

In [17]:
def get_diagvdftGrn(x):
    if pd.isna(x):
        return ['_missing']
    else:
        return [x[0], x.split('.')[0]]
    
    
########
fld = 'DiagVdftGrn'
fld_input_col = 'DiagV_val'

flg_grn_fn = get_diagvdftGrn

fld_output_col = fld_input_col.split('_')[0] + '2' + fld + '_val'
########

cols = ['P_uid', 'EC_uid', 'Diag_uid'] + [fld_input_col]
df2 = df[cols]
# df2['Diag2DiagV'] = df2['DiagV'].apply(lambda x: [x])
df2[fld_output_col] = df2[fld_input_col].apply(flg_grn_fn)
df2


# DiagV default Grains
# sent = 'I am a sentence'
# v = sent.split(' ')
# v # Sent2Token

,P_uid,EC_uid,Diag_uid,DiagV_val,DiagV2DiagVdftGrn_val
0,P1,EC11,Diag111,E11.1,"[E, E11]"
1,P1,EC12,Diag121,A11,"[A, A11]"
2,P1,EC12,Diag122,E11.1,"[E, E11]"
3,P1,EC12,Diag123,R33.3,"[R, R33]"
4,P1,EC13,None,None,[_missing]
5,P1,EC14,Diag141,F12.2,"[F, F12]"
6,P1,EC14,Diag142,E12.2,"[E, E12]"
7,P1,EC14,Diag143,E11.1,"[E, E11]"
8,P1,EC14,Diag144,G13,"[G, G13]"
9,P1,EC15,Diag151,G13,"[G, G13]"


# df_rec

In [26]:
# df['rec2grn_Vgrn'] = df['rec2grn_Vgrn'].apply(lambda x: ['_missing'] if len(x) == 0 else x)
# df_rec = df.copy()
df_rec = df2.copy()
df_rec

,P_uid,EC_uid,Diag_uid,DiagV_val,DiagV2DiagVdftGrn_val
0,P1,EC11,Diag111,E11.1,"[E, E11]"
1,P1,EC12,Diag121,A11,"[A, A11]"
2,P1,EC12,Diag122,E11.1,"[E, E11]"
3,P1,EC12,Diag123,R33.3,"[R, R33]"
4,P1,EC13,None,None,[_missing]
5,P1,EC14,Diag141,F12.2,"[F, F12]"
6,P1,EC14,Diag142,E12.2,"[E, E12]"
7,P1,EC14,Diag143,E11.1,"[E, E11]"
8,P1,EC14,Diag144,G13,"[G, G13]"
9,P1,EC15,Diag151,G13,"[G, G13]"


## build dictionaries

In [27]:
import itertools

# fld_col = fld_output

# fld_output
value_list = itertools.chain(*df_rec[fld_output_col].to_list())
d = pd.Series(value_list).value_counts().to_dict()
print(d)

v2freq = {}
v2freq['_padding'] = 0
v2freq['_missing'] = d.pop('_missing') # I should have some prefix here. 
for k, v in d.items(): v2freq[k] = v

idx2v = {idx: v for idx, v in enumerate(v2freq.keys())}
v2idx = {v:k for k, v in idx2v.items()}
print(v2idx)
print(idx2v)

{'E': 12, 'E11': 7, 'G': 5, 'G13': 5, 'F': 4, 'F12': 4, 'A': 3, 'A11': 3, 'E12': 3, 'S': 3, 'S33': 3, 'R': 2, 'R33': 2, '_missing': 2, 'E13': 2}
{'_padding': 0, '_missing': 1, 'E': 2, 'E11': 3, 'G': 4, 'G13': 5, 'F': 6, 'F12': 7, 'A': 8, 'A11': 9, 'E12': 10, 'S': 11, 'S33': 12, 'R': 13, 'R33': 14, 'E13': 15}
{0: '_padding', 1: '_missing', 2: 'E', 3: 'E11', 4: 'G', 5: 'G13', 6: 'F', 7: 'F12', 8: 'A', 9: 'A11', 10: 'E12', 11: 'S', 12: 'S33', 13: 'R', 14: 'R33', 15: 'E13'}


## map val to idx

In [28]:
# Grn here cannot be decomposed anymore. 

fld_outputidx_col = fld_output_col.replace('_val', '_idx')
df_rec[fld_outputidx_col] = df_rec[fld_output_col].apply(lambda x: [v2idx[i] for i in x])
df_rec

,P_uid,EC_uid,Diag_uid,DiagV_val,DiagV2DiagVdftGrn_val,DiagV2DiagVdftGrn_idx
0,P1,EC11,Diag111,E11.1,"[E, E11]","[2, 3]"
1,P1,EC12,Diag121,A11,"[A, A11]","[8, 9]"
2,P1,EC12,Diag122,E11.1,"[E, E11]","[2, 3]"
3,P1,EC12,Diag123,R33.3,"[R, R33]","[13, 14]"
4,P1,EC13,None,None,[_missing],[1]
5,P1,EC14,Diag141,F12.2,"[F, F12]","[6, 7]"
6,P1,EC14,Diag142,E12.2,"[E, E12]","[2, 10]"
7,P1,EC14,Diag143,E11.1,"[E, E11]","[2, 3]"
8,P1,EC14,Diag144,G13,"[G, G13]","[4, 5]"
9,P1,EC15,Diag151,G13,"[G, G13]","[4, 5]"


In [29]:

current_layer = fld_outputidx_col.split('2')[0]
print(current_layer)
parent_layer_cols = [i for i in df_rec.columns if i.split('_')[0].split('2')[0] != current_layer]
print(parent_layer_cols)

DiagV
['P_uid', 'EC_uid', 'Diag_uid']


In [32]:
cols = parent_layer_cols + [fld_outputidx_col]
print(cols)
df_rec = df_rec[cols]
df_rec

['P_uid', 'EC_uid', 'Diag_uid', 'DiagV2DiagVdftGrn_idx']


,P_uid,EC_uid,Diag_uid,DiagV2DiagVdftGrn_idx
0,P1,EC11,Diag111,"[2, 3]"
1,P1,EC12,Diag121,"[8, 9]"
2,P1,EC12,Diag122,"[2, 3]"
3,P1,EC12,Diag123,"[13, 14]"
4,P1,EC13,Diag13N,[1]
5,P1,EC14,Diag141,"[6, 7]"
6,P1,EC14,Diag142,"[2, 10]"
7,P1,EC14,Diag143,"[2, 3]"
8,P1,EC14,Diag144,"[4, 5]"
9,P1,EC15,Diag151,"[4, 5]"


In [33]:
df_rec['Diag_uid'].loc[df_rec['Diag_uid'].isna()] = df_rec['EC_uid'].loc[df_rec['Diag_uid'].isna()].apply(lambda x: x.replace('EC', 'Diag') + 'N')
df_rec

,P_uid,EC_uid,Diag_uid,DiagV2DiagVdftGrn_idx
0,P1,EC11,Diag111,"[2, 3]"
1,P1,EC12,Diag121,"[8, 9]"
2,P1,EC12,Diag122,"[2, 3]"
3,P1,EC12,Diag123,"[13, 14]"
4,P1,EC13,Diag13N,[1]
5,P1,EC14,Diag141,"[6, 7]"
6,P1,EC14,Diag142,"[2, 10]"
7,P1,EC14,Diag143,"[2, 3]"
8,P1,EC14,Diag144,"[4, 5]"
9,P1,EC15,Diag151,"[4, 5]"


# Compress Data in Different Levels

In [34]:

def get_compressed_df(level, df):
    df_rec = df
    levelid = level + '_uid'
    prefix_cols = [i for i in df_rec.columns if i[-4:] == '_uid']
    new_prefix_cols = prefix_cols[: prefix_cols.index(levelid)+1]
    old_prefix_cols = prefix_cols[prefix_cols.index(levelid)+1: ]
    # print(new_prefix_cols)
    # print(old_prefix_cols)
    df_rec = df_rec.drop(columns = old_prefix_cols)

    df_ecid = pd.DataFrame(df_rec.groupby(levelid).apply(lambda x: x.to_dict('list')).to_list())
    
    value_cols = [i for i in df_ecid.columns if i not in new_prefix_cols]
    for col in new_prefix_cols: 
        df_ecid[col] = df_ecid[col].apply(lambda x: x[0])
    # df_ecid = df_ecid.rename(columns = {k: k.replace('id', '_IX') + k.replace('id', '')  for k in old_prefix_cols})
    df_ecid = df_ecid.rename(columns = {k: f'{level}2' + k for k in df_ecid.columns if k not in new_prefix_cols})
    # df_ecid[f'{level}_L{level}'] = df_ecid.iloc[:, -1].apply(lambda x: len(x))
    return df_ecid

In [35]:
level = 'Diag'
df_diag = get_compressed_df(level, df_rec)
df_diag

,P_uid,EC_uid,Diag_uid,Diag2DiagV2DiagVdftGrn_idx
0,P1,EC11,Diag111,"[[2, 3]]"
1,P1,EC12,Diag121,"[[8, 9]]"
2,P1,EC12,Diag122,"[[2, 3]]"
3,P1,EC12,Diag123,"[[13, 14]]"
4,P1,EC13,Diag13N,[[1]]
5,P1,EC14,Diag141,"[[6, 7]]"
6,P1,EC14,Diag142,"[[2, 10]]"
7,P1,EC14,Diag143,"[[2, 3]]"
8,P1,EC14,Diag144,"[[4, 5]]"
9,P1,EC15,Diag151,"[[4, 5]]"


In [36]:
level = 'EC'
df_ec = get_compressed_df(level, df_diag)
df_ec

,P_uid,EC_uid,EC2Diag2DiagV2DiagVdftGrn_idx
0,P1,EC11,"[[[2, 3]]]"
1,P1,EC12,"[[[8, 9]], [[2, 3]], [[13, 14]]]"
2,P1,EC13,[[[1]]]
3,P1,EC14,"[[[6, 7]], [[2, 10]], [[2, 3]], [[4, 5]]]"
4,P1,EC15,"[[[4, 5]], [[2, 3]]]"
5,P1,EC16,"[[[4, 5]], [[6, 7]], [[2, 10]]]"
6,P2,EC21,"[[[11, 12]], [[4, 5]], [[2, 3]]]"
7,P2,EC22,[[[1]]]
8,P2,EC23,"[[[2, 15]]]"
9,P3,EC31,"[[[2, 3]], [[8, 9]], [[8, 9]], [[6, 7]]]"


In [37]:
level = 'P'
df_p = get_compressed_df(level, df_ec)
df_p

,P_uid,P2EC2Diag2DiagV2DiagVdftGrn_idx
0,P1,"[[[[2, 3]]], [[[8, 9]], [[2, 3]], [[13, 14]]],..."
1,P2,"[[[[11, 12]], [[4, 5]], [[2, 3]]], [[[1]]], [[..."
2,P3,"[[[[2, 3]], [[8, 9]], [[8, 9]], [[6, 7]]], [[[..."


In [38]:
full_fldgrn_name = [i for i in df_p.columns if '_idx' in i][0]
print(full_fldgrn_name)

P2EC2Diag2DiagV2DiagVdftGrn_idx


In [39]:

df_rec_p = df_rec

for layer in parent_layer_cols[::-1]:
    print(layer)
    level = layer.split('_')[0]
    df_rec_p = get_compressed_df(level, df_rec_p)
    
df_rec_p

Diag_uid
EC_uid
P_uid


,P_uid,P2EC2Diag2DiagV2DiagVdftGrn_idx
0,P1,"[[[[2, 3]]], [[[8, 9]], [[2, 3]], [[13, 14]]],..."
1,P2,"[[[[11, 12]], [[4, 5]], [[2, 3]]], [[[1]]], [[..."
2,P3,"[[[[2, 3]], [[8, 9]], [[8, 9]], [[6, 7]]], [[[..."


# Traverse

In [40]:
# from IPython.core.alias import Instance

def traverse(o, tree_types=(list, tuple), index = None, nest_layer = 100):
    if isinstance(o, tree_types) and nest_layer > 0:
        for idx, value in enumerate(o):
            new_index = index + [idx] if type(index) == list else [idx]
            for subvalue in traverse(value, tree_types, new_index, nest_layer - 1):
                yield subvalue
    else:
        if not isinstance(o, tree_types): 
            length = None
        else:
            length = len(o)
        yield index, length, o

In [42]:
full_fldgrn_name

'P2EC2Diag2DiagV2DiagVdftGrn_idx'

In [41]:
# full_fldgrn = 'P2EC2Diag2DiagV2DiagVGrn(idx)'
data = df_p[full_fldgrn_name].iloc[0]
data

[[[[2, 3]]],
 [[[8, 9]], [[2, 3]], [[13, 14]]],
 [[[1]]],
 [[[6, 7]], [[2, 10]], [[2, 3]], [[4, 5]]],
 [[[4, 5]], [[2, 3]]],
 [[[4, 5]], [[6, 7]], [[2, 10]]]]

In [43]:
layer_num = len(full_fldgrn_name.split('2')) - 1
print(layer_num)

4


In [37]:
# data[2][0]

In [38]:
a = list(traverse(data, nest_layer = 0))
# length_info = np.array([i[0] for i in a])
# length_info
# a = [(i[0], len(i[1])) for i in a]
# a
a

[(None, 4, [[[[5, 6]]], [[[1]]], [[[2, 3]], [[2, 3]]], [[[2, 4]], [[1]]]])]

In [45]:
data

[[[[2, 3]]],
 [[[8, 9]], [[2, 3]], [[13, 14]]],
 [[[1]]],
 [[[6, 7]], [[2, 10]], [[2, 3]], [[4, 5]]],
 [[[4, 5]], [[2, 3]]],
 [[[4, 5]], [[6, 7]], [[2, 10]]]]

In [44]:
a = list(traverse(data, nest_layer = 1))
# length_info = np.array([i[0] for i in a])
# length_info
# a = [(i[0], len(i[1])) for i in a]
# a
a

[([0], 1, [[[2, 3]]]),
 ([1], 3, [[[8, 9]], [[2, 3]], [[13, 14]]]),
 ([2], 1, [[[1]]]),
 ([3], 4, [[[6, 7]], [[2, 10]], [[2, 3]], [[4, 5]]]),
 ([4], 2, [[[4, 5]], [[2, 3]]]),
 ([5], 3, [[[4, 5]], [[6, 7]], [[2, 10]]])]

In [47]:
data[1]

[[[8, 9]], [[2, 3]], [[13, 14]]]

In [48]:
a = list(traverse(data, nest_layer = 2))
# length_info = np.array([i[0] for i in a])
# length_info
# a = [(i[0], len(i[1])) for i in a]
# a
a

[([0, 0], 1, [[2, 3]]),
 ([1, 0], 1, [[8, 9]]),
 ([1, 1], 1, [[2, 3]]),
 ([1, 2], 1, [[13, 14]]),
 ([2, 0], 1, [[1]]),
 ([3, 0], 1, [[6, 7]]),
 ([3, 1], 1, [[2, 10]]),
 ([3, 2], 1, [[2, 3]]),
 ([3, 3], 1, [[4, 5]]),
 ([4, 0], 1, [[4, 5]]),
 ([4, 1], 1, [[2, 3]]),
 ([5, 0], 1, [[4, 5]]),
 ([5, 1], 1, [[6, 7]]),
 ([5, 2], 1, [[2, 10]])]

In [50]:
idx_list = [3, 0]
tmp = data
for idx in idx_list:
    tmp = tmp[idx]
tmp

[[6, 7]]

In [51]:
a = list(traverse(data, nest_layer = 3))
# length_info = np.array([i[0] for i in a])
# length_info
# a = [(i[0], len(i[1])) for i in a]
# a
a

[([0, 0, 0], 2, [2, 3]),
 ([1, 0, 0], 2, [8, 9]),
 ([1, 1, 0], 2, [2, 3]),
 ([1, 2, 0], 2, [13, 14]),
 ([2, 0, 0], 1, [1]),
 ([3, 0, 0], 2, [6, 7]),
 ([3, 1, 0], 2, [2, 10]),
 ([3, 2, 0], 2, [2, 3]),
 ([3, 3, 0], 2, [4, 5]),
 ([4, 0, 0], 2, [4, 5]),
 ([4, 1, 0], 2, [2, 3]),
 ([5, 0, 0], 2, [4, 5]),
 ([5, 1, 0], 2, [6, 7]),
 ([5, 2, 0], 2, [2, 10])]

In [52]:
a = list(traverse(data, nest_layer = 4))
# length_info = np.array([i[0] for i in a])
# length_info
# a = [(i[0], len(i[1])) for i in a]
# a
a

[([0, 0, 0, 0], None, 2),
 ([0, 0, 0, 1], None, 3),
 ([1, 0, 0, 0], None, 8),
 ([1, 0, 0, 1], None, 9),
 ([1, 1, 0, 0], None, 2),
 ([1, 1, 0, 1], None, 3),
 ([1, 2, 0, 0], None, 13),
 ([1, 2, 0, 1], None, 14),
 ([2, 0, 0, 0], None, 1),
 ([3, 0, 0, 0], None, 6),
 ([3, 0, 0, 1], None, 7),
 ([3, 1, 0, 0], None, 2),
 ([3, 1, 0, 1], None, 10),
 ([3, 2, 0, 0], None, 2),
 ([3, 2, 0, 1], None, 3),
 ([3, 3, 0, 0], None, 4),
 ([3, 3, 0, 1], None, 5),
 ([4, 0, 0, 0], None, 4),
 ([4, 0, 0, 1], None, 5),
 ([4, 1, 0, 0], None, 2),
 ([4, 1, 0, 1], None, 3),
 ([5, 0, 0, 0], None, 4),
 ([5, 0, 0, 1], None, 5),
 ([5, 1, 0, 0], None, 6),
 ([5, 1, 0, 1], None, 7),
 ([5, 2, 0, 0], None, 2),
 ([5, 2, 0, 1], None, 10)]

In [54]:
# idx, length, values = a[1] 
# idx = tuple(idx)


idx =  [5, 2, 0, 1] #
final = data.copy()
for i in idx:
    final = final[i]
final

10

# Turn to PyTorch Dataloader

In [55]:
print(full_fldgrn_name)

P2EC2Diag2DiagV2DiagVdftGrn_idx


In [56]:
# data points collate_fn, first back to df.
a, b = full_fldgrn_name.split('2')[:2]
# print(a, b)

len_name = f'{a}_ln{b}'
print(len_name)

df_batch = df_p.copy()
df_batch[len_name] = df_batch[full_fldgrn_name].apply(len)
df_batch = df_batch.sort_values(len_name, ascending = False)
df_batch

P_lnEC


,P_uid,P2EC2Diag2DiagV2DiagVdftGrn_idx,P_lnEC
0,P1,"[[[[2, 3]]], [[[8, 9]], [[2, 3]], [[13, 14]]],...",6
2,P3,"[[[[2, 3]], [[8, 9]], [[8, 9]], [[6, 7]]], [[[...",5
1,P2,"[[[[11, 12]], [[4, 5]], [[2, 3]]], [[[1]]], [[...",3


In [57]:
values_list = df_batch[full_fldgrn_name].to_list()
values_list

[[[[[2, 3]]],
  [[[8, 9]], [[2, 3]], [[13, 14]]],
  [[[1]]],
  [[[6, 7]], [[2, 10]], [[2, 3]], [[4, 5]]],
  [[[4, 5]], [[2, 3]]],
  [[[4, 5]], [[6, 7]], [[2, 10]]]],
 [[[[2, 3]], [[8, 9]], [[8, 9]], [[6, 7]]],
  [[[11, 12]]],
  [[[4, 5]], [[2, 10]], [[13, 14]]],
  [[[11, 12]], [[6, 7]]],
  [[[2, 3]], [[2, 15]]]],
 [[[[11, 12]], [[4, 5]], [[2, 3]]], [[[1]]], [[[2, 15]]]]]

In [58]:
o = values_list
idx = 4
output = list(traverse(o, nest_layer = idx))
output

[([0, 0, 0, 0], 2, [2, 3]),
 ([0, 1, 0, 0], 2, [8, 9]),
 ([0, 1, 1, 0], 2, [2, 3]),
 ([0, 1, 2, 0], 2, [13, 14]),
 ([0, 2, 0, 0], 1, [1]),
 ([0, 3, 0, 0], 2, [6, 7]),
 ([0, 3, 1, 0], 2, [2, 10]),
 ([0, 3, 2, 0], 2, [2, 3]),
 ([0, 3, 3, 0], 2, [4, 5]),
 ([0, 4, 0, 0], 2, [4, 5]),
 ([0, 4, 1, 0], 2, [2, 3]),
 ([0, 5, 0, 0], 2, [4, 5]),
 ([0, 5, 1, 0], 2, [6, 7]),
 ([0, 5, 2, 0], 2, [2, 10]),
 ([1, 0, 0, 0], 2, [2, 3]),
 ([1, 0, 1, 0], 2, [8, 9]),
 ([1, 0, 2, 0], 2, [8, 9]),
 ([1, 0, 3, 0], 2, [6, 7]),
 ([1, 1, 0, 0], 2, [11, 12]),
 ([1, 2, 0, 0], 2, [4, 5]),
 ([1, 2, 1, 0], 2, [2, 10]),
 ([1, 2, 2, 0], 2, [13, 14]),
 ([1, 3, 0, 0], 2, [11, 12]),
 ([1, 3, 1, 0], 2, [6, 7]),
 ([1, 4, 0, 0], 2, [2, 3]),
 ([1, 4, 1, 0], 2, [2, 15]),
 ([2, 0, 0, 0], 2, [11, 12]),
 ([2, 0, 1, 0], 2, [4, 5]),
 ([2, 0, 2, 0], 2, [2, 3]),
 ([2, 1, 0, 0], 1, [1]),
 ([2, 2, 0, 0], 2, [2, 15])]

# Convert to Numpy

In [59]:
o = values_list

new_full_fldgrn_name = 'B2' + full_fldgrn_name
print(new_full_fldgrn_name)

layer_num = len(new_full_fldgrn_name.split('2'))
layers = new_full_fldgrn_name.replace('_idx', '').split('2')
# L = [len(values_list)] 


D = {}

idx = 0
layer_parents = layers[:idx + 1]
layer_children = layers[idx + 1]
len_name = f'{"2".join(layer_parents)}_ln{layer_children}'
len_np = np.array(len(values_list))
len_shapes = [len_np.max()] # from layer 0, prepare for layer 1. 

D[len_name] = len_np

for idx in range(1, layer_num - 1):
    output = list(traverse(o, nest_layer = idx))
    # print(output)
    # data = np.zeros(L)
    print('\n\n')
    print(idx)
    print(output)

    layer_parents = layers[:idx + 1]
    layer_children = layers[idx + 1]
    len_name = f'{"2".join(layer_parents)}_ln{layer_children}'
    print(len_name)
    
    locidx  = [i[0] for i in output]
    length = [i[1] for i in output]
    # values = [i[2] for i in output]

    len_np = np.zeros(len_shapes).astype(int)
    print(len_np.shape, '<---- len_np.shape')
    for locidx, length, _ in output:
        len_np[tuple(locidx)] = int(length)
    print(len_np)
    len_shapes.append(len_np.max())
    print(len_shapes, '<---- next len_np.shape')
    print(length)
    # print()
    D[len_name] = len_np


idx = layer_num
name = new_full_fldgrn_name
data = np.zeros(len_shapes) # don't convert it to int for now. 
output = list(traverse(o, nest_layer = idx))
for locidx, _, value in output:
    data[tuple(locidx)] = value
    # print(locidx, value)
    # print(data[tuple(locidx)])# = value
data.shape
D[name] = data


B2P2EC2Diag2DiagV2DiagVdftGrn_idx



1
[([0], 6, [[[[2, 3]]], [[[8, 9]], [[2, 3]], [[13, 14]]], [[[1]]], [[[6, 7]], [[2, 10]], [[2, 3]], [[4, 5]]], [[[4, 5]], [[2, 3]]], [[[4, 5]], [[6, 7]], [[2, 10]]]]), ([1], 5, [[[[2, 3]], [[8, 9]], [[8, 9]], [[6, 7]]], [[[11, 12]]], [[[4, 5]], [[2, 10]], [[13, 14]]], [[[11, 12]], [[6, 7]]], [[[2, 3]], [[2, 15]]]]), ([2], 3, [[[[11, 12]], [[4, 5]], [[2, 3]]], [[[1]]], [[[2, 15]]]])]
B2P_lnEC
(3,) <---- len_np.shape
[6 5 3]
[3, 6] <---- next len_np.shape
3



2
[([0, 0], 1, [[[2, 3]]]), ([0, 1], 3, [[[8, 9]], [[2, 3]], [[13, 14]]]), ([0, 2], 1, [[[1]]]), ([0, 3], 4, [[[6, 7]], [[2, 10]], [[2, 3]], [[4, 5]]]), ([0, 4], 2, [[[4, 5]], [[2, 3]]]), ([0, 5], 3, [[[4, 5]], [[6, 7]], [[2, 10]]]), ([1, 0], 4, [[[2, 3]], [[8, 9]], [[8, 9]], [[6, 7]]]), ([1, 1], 1, [[[11, 12]]]), ([1, 2], 3, [[[4, 5]], [[2, 10]], [[13, 14]]]), ([1, 3], 2, [[[11, 12]], [[6, 7]]]), ([1, 4], 2, [[[2, 3]], [[2, 15]]]), ([2, 0], 3, [[[11, 12]], [[4, 5]], [[2, 3]]]), ([2, 1], 1, [[[1

In [62]:
info_idx = D['B2P2EC2Diag2DiagV2DiagVdftGrn_idx']
info_idx.shape

(3, 6, 4, 1, 2)

In [60]:
for k, v in D.items():
    print(k, v.shape)

B_lnP ()
B2P_lnEC (3,)
B2P2EC_lnDiag (3, 6)
B2P2EC2Diag_lnDiagV (3, 6, 4)
B2P2EC2Diag2DiagV_lnDiagVdftGrn (3, 6, 4, 1)
B2P2EC2Diag2DiagV2DiagVdftGrn_idx (3, 6, 4, 1, 2)


# Function

In [63]:
def convert_relational_list_to_numpy(values_list, new_full_fldgrn_name):
    
    o = values_list
    layer_num = len(new_full_fldgrn_name.split('2'))
    layers = new_full_fldgrn_name.replace('_idx', '').split('2')
    # L = [len(values_list)] 

    D = {}

    # (1) from first layer: 0
    idx = 0
    layer_parents = layers[:idx + 1]
    layer_children = layers[idx + 1]
    len_name = f'{"2".join(layer_parents)}_ln{layer_children}'
    len_np = np.array(len(values_list))
    len_shapes = [len_np.max()] # from layer 0, prepare for layer 1. 
    D[len_name] = len_np

    # (2) from 1 - last one layers
    for idx in range(1, layer_num - 1):
        output = list(traverse(o, nest_layer = idx))
        # print(output)
        # data = np.zeros(L)
        # print('\n\n')
        # print(idx)
        # print(output)

        layer_parents = layers[:idx + 1]
        layer_children = layers[idx + 1]
        len_name = f'{"2".join(layer_parents)}_ln{layer_children}'
        # print(len_name)

        locidx  = [i[0] for i in output]
        length = [i[1] for i in output]
        # values = [i[2] for i in output]

        len_np = np.zeros(len_shapes).astype(int)
        # print(len_np.shape, '<---- len_np.shape')
        for locidx, length, _ in output:
            len_np[tuple(locidx)] = int(length)
        # print(len_np)
        len_shapes.append(len_np.max())
        # print(len_shapes, '<---- next len_np.shape')
        # print(length)
        # print()
        D[len_name] = len_np

    # (3) for the data
    idx = layer_num
    name = new_full_fldgrn_name
    data = np.zeros(len_shapes) # don't convert it to int for now. 
    output = list(traverse(o, nest_layer = idx))
    for locidx, _, value in output:
        data[tuple(locidx)] = value
        # print(locidx, value)
        # print(data[tuple(locidx)])# = value
    data.shape
    D[name] = data
    
    return D


In [64]:
print(full_fldgrn_name)

P2EC2Diag2DiagV2DiagVdftGrn_idx


In [65]:
# data points collate_fn, first back to df.
a, b = full_fldgrn_name.split('2')[:2]
# print(a, b)

len_name = f'{a}_ln{b}'
print(len_name)

df_batch = df_p.copy()
df_batch[len_name] = df_batch[full_fldgrn_name].apply(len)
df_batch = df_batch.sort_values(len_name, ascending = False)
df_batch

P_lnEC


,P_uid,P2EC2Diag2DiagV2DiagVdftGrn_idx,P_lnEC
0,P1,"[[[[2, 3]]], [[[8, 9]], [[2, 3]], [[13, 14]]],...",6
2,P3,"[[[[2, 3]], [[8, 9]], [[8, 9]], [[6, 7]]], [[[...",5
1,P2,"[[[[11, 12]], [[4, 5]], [[2, 3]]], [[[1]]], [[...",3


In [66]:
values_list = df_batch[full_fldgrn_name].to_list()
new_full_fldgrn_name = 'B2' + full_fldgrn_name
print(new_full_fldgrn_name)
D = convert_relational_list_to_numpy(values_list, new_full_fldgrn_name)
for k, v in D.items(): print(k, v.shape)

B2P2EC2Diag2DiagV2DiagVdftGrn_idx
B_lnP ()
B2P_lnEC (3,)
B2P2EC_lnDiag (3, 6)
B2P2EC2Diag_lnDiagV (3, 6, 4)
B2P2EC2Diag2DiagV_lnDiagVdftGrn (3, 6, 4, 1)
B2P2EC2Diag2DiagV2DiagVdftGrn_idx (3, 6, 4, 1, 2)


In [67]:
keys = [i for i in D]
values = [v for k, v in D.items()]
layers = [i for i in range(len(D))]

df = pd.DataFrame({'layer': layers, 'name': keys, 'value': values})
df.to_pickle('df_batch_np.p')

# Test the Results

In [68]:
D_values = [v for k, v in D.items()]
data = D_values[-1]
data.shape

(3, 6, 4, 1, 2)

In [70]:
name = 'B2P2EC2Diag2DiagV2DiagVdftGrn_idx'

In [72]:
info_idx = D[name]
info_idx.shape

#(3,6, 4,   1,    2)
'B2P2EC2Diag2DiagV2DiagVdftGrn_idx'

(3, 6, 4, 1, 2)

In [ ]:
#(128, 73)
'B2Sent2Token_idx'

In [73]:
# data[0, 0, 0]

In [56]:
df_rec

,P_uid,EC_uid,Diag_uid,DiagV2DiagVdftGrn_idx
0,1,1_1,1_1_1,"[5, 6]"
1,1,1_2,1_2_1,[1]
2,1,1_3,1_3_1,"[2, 3]"
3,1,1_3,1_3_2,"[2, 3]"
4,1,1_4,1_4_1,"[2, 4]"
5,1,1_4,1_4_2,[1]
6,2,2_1,2_1_1,"[2, 4]"
7,2,2_2,2_2_1,[1]
8,2,2_2,2_2_2,[1]
9,2,2_3,2_3_1,[1]


In [87]:
idx2v

{0: '_padding',
 1: '_missing',
 2: 'E',
 3: 'E11',
 4: 'F',
 5: 'F12',
 6: 'R',
 7: 'R33',
 8: 'E13',
 9: 'E12',
 10: 'G',
 11: 'G13'}

# --- Parking

In [ ]:
def get_batch_info(df_batch):
    df_batch = df_batch.sort_values(df_batch.columns[-1], ascending = False)
    value_cols = [i for i in df_batch.columns if '_L' in i or '_I' in i and '_IX' not in i]
    value_cols.reverse()
    L = [len(df_batch)]
    d = {}
    Data = {}
    
    for idx, L_col in enumerate(value_cols):
        # print('\nL is:', L_col, L)
        d[L_col] = L.copy()
        o = df_batch[L_col].to_list()

        # print(L)
        output = list(traverse(o, nest_layer = idx))
        # print(output)
        data = np.zeros(L)

        if idx == 0:
            locidx, v = output[0]
            data[:len(v)] = v
        else:
            for idx, loc_v in enumerate(output):
                locidx, v = loc_v
                data[tuple(locidx)][:len(v)] = v

        Data[L_col] = data
        p = int(data.max())
        # p = max(traverse(o))
        L.append(p)

    return Data

In [ ]:
values_list = df_batch[full_fldgrn]

print(len(values_list))

3


In [ ]:
Data = get_batch_info(df_batch)
[i for i in Data]

['p_Lp', 'p2ec_Lec', 'p2ec2rec_Lrec', 'p2ec2rec2grn_Igrn']

In [ ]:
for k, v in Data.items(): print(k, v)

p_Lp [6. 4. 4.]
p2ec_Lec [[2. 1. 2. 2. 2. 2.]
 [2. 2. 2. 1. 0. 0.]
 [2. 2. 1. 1. 0. 0.]]
p2ec2rec_Lrec [[[8. 8.]
  [7. 0.]
  [8. 2.]
  [3. 3.]
  [6. 1.]
  [3. 4.]]

 [[2. 7.]
  [3. 6.]
  [6. 5.]
  [5. 0.]
  [0. 0.]
  [0. 0.]]

 [[4. 8.]
  [1. 4.]
  [5. 0.]
  [7. 0.]
  [0. 0.]
  [0. 0.]]]
p2ec2rec2grn_Igrn [[[[1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 1. 1. 1. 1. 1. 1.]]

  [[1. 1. 1. 1. 1. 1. 1. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0.]]

  [[1. 1. 1. 1. 1. 1. 1. 1.]
   [1. 1. 0. 0. 0. 0. 0. 0.]]

  [[1. 1. 1. 0. 0. 0. 0. 0.]
   [1. 1. 1. 0. 0. 0. 0. 0.]]

  [[1. 1. 1. 1. 1. 1. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0.]]

  [[1. 1. 1. 0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 0. 0. 0. 0.]]]


 [[[1. 1. 0. 0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1. 1. 1. 0.]]

  [[1. 1. 1. 0. 0. 0. 0. 0.]
   [1. 1. 1. 1. 1. 1. 0. 0.]]

  [[1. 1. 1. 1. 1. 1. 0. 0.]
   [1. 1. 1. 1. 1. 0. 0. 0.]]

  [[1. 1. 1. 1. 1. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0. 0. 0. 0.]
  

In [ ]:
df_batch = df_batch.sort_values(df_batch.columns[-1], ascending = False)
df_batch

,pid,p2ec_IXec,p2ec2rec_IXrec,p2ec2rec2grn_Vgrn,p2ec2rec2grn_Igrn,p2ec2rec_Lrec,p2ec_Lec,p_Lp
0,1,"[1_1, 1_2, 1_3, 1_4, 1_5]","[[1_1_1, 1_1_2], [1_2_1, 1_2_2], [1_3_1], [1_4...","[[[aa, aa, aa, aa, aa, aa], [aa, aa, aa, aa, a...","[[[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]], [[1, 1...","[[6, 5], [8, 4], [5], [5], [4]]","[2, 2, 1, 1, 1]",5
2,3,"[3_1, 3_2, 3_3, 3_4, 3_5]","[[3_1_1, 3_1_2], [3_2_1, 3_2_2], [3_3_1], [3_4...","[[[aa, aa, aa, aa, aa, aa, aa], [aa, aa, aa, a...","[[[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]], ...","[[7, 6], [6, 8], [7], [6], [7, 1]]","[2, 2, 1, 1, 2]",5
1,2,"[2_1, 2_2, 2_3]","[[2_1_1, 2_1_2], [2_2_1, 2_2_2], [2_3_1, 2_3_2]]","[[[aa, aa, aa, aa, aa], [aa, aa, aa, aa]], [[a...","[[[1, 1, 1, 1, 1], [1, 1, 1, 1]], [[1, 1, 1], ...","[[5, 4], [3, 1], [1, 8]]","[2, 2, 2]",3


In [ ]:
176 + 299

475

In [ ]:
# # data points collate_fn, first back to df.


# full_fldgrn_name = 'P2EC2Diag2DiagV2DiagVGrn(idx)'

# a, b = full_fldgrn_name.split('2')[:2]
# # print(a, b)

# len_name = f'{a}_Ln{b}'
# len_name

In [ ]:
# def traverse(o, tree_types=(list, tuple), index = None, nest_layer = 100):
#     # if nest_layer == 0:
#     #     # index = [str(i) for i in index]
#     #     # yield '-'.join(index), o 
#     #     yield index, o 
#     # else:
#     if isinstance(o, tree_types) and nest_layer > 0:
#         for idx, value in enumerate(o):
#             new_index = index + [idx] if type(index) == list else [idx]
#             for subvalue in traverse(value, tree_types, new_index, nest_layer - 1):
#                 yield subvalue
#     else:
#         # index = [str(i) for i in index]
#         # index = [str(i) for i in index]
#         # yield '-'.join(index), o 
#         # index = tuple(index) if type(index) == list else 0
#         yield index, o 

In [ ]:
# def get_batch_info(values_list, full_fldgrn):
#     o = values_list
#     layer_num = len(full_fldgrn.split('2')) - 1

#     L = [len(values_list)] 
#     for idx in range(layer_num):
#         # print('\nL is:', L_col, L)
#         # d[L_col] = L.copy()
#         # o = df_batch[L_col].to_list()

#         # print(L)
#         output = list(traverse(o, nest_layer = idx))
#         # print(output)
#         data = np.zeros(L)

#         if idx == 0:
#             locidx, v = output[0]
#             data[:len(v)] = v
#         else:
#             for idx, loc_v in enumerate(output):
#                 locidx, v = loc_v
#                 data[tuple(locidx)][:len(v)] = v

#         # Data[L_col] = data
#         p = int(data.max())
#         # p = max(traverse(o))
#         L.append(p)

#     return data